<a href="https://colab.research.google.com/github/rthederra/Hito-2---Gestion-Financiera/blob/main/valoraci%C3%B3n_de_opciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from scipy.stats import norm
import numpy as np
import pandas as pd

!pip install bolsa-stgo

# Valoración opciones aplicada

Tome en consideración el modelo continuio de Black and Scholes:

$$C(S,t)=S\cdot\Phi(d_1)-K\exp(-R\cdot t)\cdot\Phi(d_2)$$

donde $d_1=\frac{\log\frac{S}{K}+\left(R+\frac{\sigma^2}{2}\right)\cdot t}{\sqrt{\sigma^2\cdot t}}$ y $d_2=d_1-\sqrt{\sigma^2\cdot t}$.

Defina los parámetros asociados a valores del activo definido en la Tarea 1 y calcule el valor de una opción de compra.

## Black-Scholes

In [ ]:
# Parámetros de test
K = 100
r = 0.02
sigma = 0.2
T = 0.5
S0 = 102
# Resultado esperado CALL
# 7.28815118277163

d1 = (math.log(S0 / K) + (r + (sigma**2) / 2) * T) / (sigma * math.sqrt(T))
d2 = d1 - sigma * math.sqrt(T)
bs_call = S0 * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
bs_put = K * math.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

print("opcion de compra (call): ", bs_call)
print("opcion de venta (put): ", bs_put)

opcion de compra (call):  7.288151182771628
opcion de venta (put):  4.2931345576884326


## Segundo Modelo de valoración

###Aproximación binomial
El modelo binomial es un enfoque discreto que divide el tiempo hasta la expiración en pasos discretos y modela el comportamiento del precio del activo subyacente como una serie de movimientos hacia arriba y hacia abajo con sus respectivas probabilidades. Este modelo puede utilizarse para valorar opciones de compra y venta.

A continuación esta el codigo de el modelo binomial con 1.000 pasos:

In [ ]:
def aprox_binomial(S0, K, T, r, sigma, n, opcion='call'):

    dt = T / n                                                                                                          #paso del tiempo, tiempo total/ n pasos
    u = math.exp(sigma * math.sqrt(dt))                                                                                 #crecimiento del precio del activo subyacente
    d = 1 / u                                                                                                           #decrecimiento del precio del activo subyacente
    q = (math.exp(r * dt) - d) / (u - d)                                                                                #probabilidad de que el precio del activo subyacente suba a lo largo del tiempo

                                                                                                                        #construir el árbol binomial para el precio del activo subyacente
    arbol_precios = [[0 for _ in range(n+1)] for _ in range(n+1)]                                                       #almacena los posibles precios del activo subyacente en cada punto del tiempo
    for j in range(n+1):
        arbol_precios[n][j] = S0 * (u ** (n - j)) * (d ** j)
                                                                                                                        #calcular el valor de la opción utilizando el árbol binomial
    arbol_opciones = [[0 for _ in range(n+1)] for _ in range(n+1)]                                                      #almacena el valor de la opción en los mismos puntos que el arbol de precios
    for j in range(n+1):
        if opcion == 'call':
            arbol_opciones[n][j] = max(0, arbol_precios[n][j] - K)
        elif opcion == 'put':
            arbol_opciones[n][j] = max(0, K - arbol_precios[n][j])
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            if opcion == 'call':
                arbol_opciones[i][j] = math.exp(-r * dt) * (q * arbol_opciones[i+1][j] + (1 - q) * arbol_opciones[i+1][j+1])
            elif opcion == 'put':
                arbol_opciones[i][j] = math.exp(-r * dt) * (q * arbol_opciones[i+1][j] + (1 - q) * arbol_opciones[i+1][j+1])
    return arbol_opciones[0][0]

n = 1000                                                                                                                #numero de pasos en la aproximación binomial
opcion = 'call'
precio = aprox_binomial(S0, K, T, r, sigma, n, opcion)
print("el precio de la opcion de compra (call) es: ", precio)
opcion = 'put'
precio = aprox_binomial(S0, K, T, r, sigma, n, opcion)
print("el precio de la opcion de venta (put) es: ", precio)

el precio de la opcion de compra (call) es:  7.288615305151679
el precio de la opcion de venta (put) es:  4.293598680078453


Podriamos tratar de hacerlo con más pasos para darle más exactitud, por ejemplo 10.000 pero demorará mucho más y no cambia mucho el resultado. La aproximación es casi la misma

In [ ]:
n = 10000
opcion = 'call'
precio = aprox_binomial(S0, K, T, r, sigma, n, opcion)
print("el precio de la opcion de compra (call) es: ", precio)
opcion = 'put'
precio = aprox_binomial(S0, K, T, r, sigma, n, opcion)
print("el precio de la opcion de venta (put) es: ", precio)

el precio de la opcion de compra (call) es:  7.288013408812326
el precio de la opcion de venta (put) es:  4.292996783767374


El modelo binomial es una aproximación discreta, que divide al tiempo en pasos discretos, es decir, el precio se modela en base a la probabilidad de crecimiento/decrecimiento de este, por lo que es bueno para activos que fluctúan mucho y que pueden dar saltos grandes. Una desventaja de este modelo es que puede ser muy pesado para una computadora, por la cantidad grande de pasos.

El modelo de Black-Scholes es una aproximación continua que usa ecuaciones diferenciales estocásticas para aproximar el precio, trabajando con formulas analíticas para calcular el precio teórico de las opciones, por lo que no utiliza tiempos discretos como el modelo binomial. Una desventaja de este modelo es que no tiene en cuenta movimientos abruptos del precio de las opciones y requiere suposiciones sobre lo constante de la volatilidad y de las tasas de interés, que no siempre es así.

Como podemos ver en los codigos de arriba, mediante mayor cantidad de pasos en el modelo binomial mayor exactitud, pero podemos decir que ambos precios que nos otorga este modelo (en n=1.000 y n=10.000) fluctuan al rededor de el precio calculado con Black-Schols, por lo que podriamos decir que este modelo es el más acertado en este caso.


## Ingesta de datos

In [ ]:
!pip install yfinance
from datetime import datetime
import yfinance as yf

Utilizamos la empresa de Estados Unidos American Tower Corporation, debido a que no se encontró inmobiliarias chilenas que tuvieran opciones.

In [ ]:
# Define el símbolo de la empresa y crea un objeto para obtener datos de opciones
empresa = "AMT"  # Cambia esto al símbolo de la empresa que te interesa
opcion = yf.Ticker(f"{empresa}")

# Obtiene la información de las opciones
opciones = opcion.options
# Muestra las fechas de vencimiento de las opciones disponibles
fecha = opciones[5]
print("fecha que se va a utilizar: ", fecha)
#print("Fechas de vencimiento de las opciones disponibles:", opciones)

fecha que se va a utilizar:  2025-01-17


Con la fecha de vencimiento de la opción tenemos acceso a sus datos, S0, K, T y sigma. Utilizaremos una tasas de 2%.

In [ ]:
fecha_vencimiento = fecha
opciones_detalle = opcion.option_chain(fecha_vencimiento)
fecha_vencimiento = datetime(2023, 11, 10)
#print(opciones_detalle)
#print(opciones_detalle.calls.head())
S0 = opciones_detalle.calls['lastPrice'].iloc[0]
K = opciones_detalle.calls['strike'].iloc[0]
T = (fecha_vencimiento - datetime.today()).days / 365.0
sigma = opciones_detalle.calls['impliedVolatility'].iloc[0]
r = 0.02
print("precio actual: ", S0, "\nstrike: ", K, "\ntiempo: ",T, "\nsigma: ", sigma)

precio actual:  96.5 
strike:  95.0 
tiempo:  0.03561643835616438 
sigma:  0.8428360443115235


###Black-Scholes

In [ ]:
d1 = (math.log(S0 / K) + (r + (sigma**2) / 2) * T) / (sigma * math.sqrt(T))
d2 = d1 - sigma * math.sqrt(T)
bs_call = S0 * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
bs_put = K * math.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

print("opcion de compra (call): ", bs_call)
print("opcion de venta (put): ", bs_put)

opcion de compra (call):  6.883340312323192
opcion de venta (put):  5.315693175807574


###Modelo Binomial

In [ ]:
n = 1000
opcion = 'call'
precio = aprox_binomial(S0, K, T, r, sigma, n, opcion)
print("el precio de la opcion de compra (call) es: ", precio)
opcion = 'put'
precio = aprox_binomial(S0, K, T, r, sigma, n, opcion)
print("el precio de la opcion de venta (put) es: ", precio)

el precio de la opcion de compra (call) es:  6.884799712232387
el precio de la opcion de venta (put) es:  5.317152575711843
